# UR5 Install

This driver requires a system setup with ROS. It is recommended to use Ubuntu 18.04 with ROS melodic, however using Ubuntu 16.04 with ROS kinetic should also work.

The driver consists of a C++ library that abstracts the robot's interfaces and a ROS driver on top of that.

https://github.com/UniversalRobots/Universal_Robots_ROS_Driver



The instructions to build our workspace are (substitute "catkin_ws" by your workspace name):

In [ ]:
# create a catkin workspace
$ mkdir -p catkin_ws/src && cd catkin_ws

# clone the driver
$ git clone https://github.com/UniversalRobots/Universal_Robots_ROS_Driver.git src/Universal_Robots_ROS_Driver

# clone fork of the description. This is currently necessary, until the changes are merged upstream.
$ git clone -b calibration_devel https://github.com/fmauch/universal_robot.git src/fmauch_universal_robot

# install dependencies
$ sudo apt update -qq
$ rosdep update
$ rosdep install --from-paths src --ignore-src -y

# build the workspace. We need an isolated build because of the non-catkin library package.
$ catkin_make

# activate the workspace (ie: source it)
$ source devel/setup.bash

Verify the lasts lines in .bashrc file

In [ ]:
source /opt/ros/melodic/setup.bash
source ~/ur5arm_ws/devel/setup.bash
export ROS_IP=192.168.18.92
export ROS_MASTER_URI=http://localhost:11311


## Prepare the robot
For using the ur_robot_driver with a real robot you need to install the externalcontrol-1.0.4.urcap which can be found inside the resources folder of this driver.

## Prepare the ROS PC
Each UR robot is calibrated inside the factory giving exact forward and inverse kinematics. To also make use of this in ROS, you first have to extract the calibration information from the robot.

Though this step is not necessary to control the robot using this driver, it is highly recommended to do so, as otherwise endeffector positions might be off in the magnitude of centimeters.

### Quick start
If you calibrated your robot before, pass that calibration to the launch file:

## UR5 simulation

###  in Gazebo

There are launch files available to bringup a simulated robot in Gazebo.

To bring up the simulated robot in Gazebo, run:

In [ ]:
roslaunch ur_gazebo ur5.launch

In [ ]:
rosrun rqt_joint_trajectory_controller rqt_joint_trajectory_controller

### MoveIt! with a simulated robot

You can use MoveIt! to control the simulated robot.


https://github.com/ros-industrial/universal_robot/tree/melodic-devel

For setting up the MoveIt! nodes to allow motion planning run:

In [ ]:
roslaunch ur5_moveit_config ur5_moveit_planning_execution.launch sim:=true

<img src="./Images/1_ur5_gazebo1.png">

For starting up RViz with a configuration including the MoveIt! Motion Planning plugin run:

In [ ]:
roslaunch ur5_moveit_config moveit_rviz.launch config:=true

# MoveIt!
Tutorial
http://docs.ros.org/en/melodic/api/moveit_tutorials/html/index.html

Videos
https://www.youtube.com/watch?v=b4T577d39dE
https://www.youtube.com/watch?v=BxCik8OI1Fw

Gamepad Xbox360:
https://www.amazon.es/dp/B0886GTMM2/ref=sspa_dk_detail_0?psc=1&pd_rd_i=B0886GTMM2&pd_rd_w=FyS9h&pf_rd_p=b39a6510-b997-4445-a1c4-cee175eeff20&pd_rd_wg=aDvTo&pf_rd_r=TAJCTPY4B5ZPF5HH6X9D&pd_rd_r=5774e3e1-ce0c-4005-8669-3cd1d7738859&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUFYNjBFUFJFM0dJMFAmZW5jcnlwdGVkSWQ9QTAxMzQ4NjgxTDk5SDMzVlJHRzNMJmVuY3J5cHRlZEFkSWQ9QTA5MzAzNDQyVFVLTUMzNktISlFHJndpZGdldE5hbWU9c3BfZGV0YWlsJmFjdGlvbj1jbGlja1JlZGlyZWN0JmRvTm90TG9nQ2xpY2s9dHJ1ZQ==


### Kinematics control

We create a ur5control package with dependencies:
moveit_core moveit_visual_tools moveit_ros_planning_interface interactive_markers tf2_geometry_msgs

In [ ]:
catkin_create_pkg ur5control rospy std_msgs moveit_core moveit_visual_tools moveit_ros_planning_interface interactive_markers tf2_geometry_msgs

Open moveIt! and verify the name of move_group=manipulator

## UR5 real



### Connection

We connect UR5 and PC with Ethernet cable

Assign IP static address:

192.168.1.1 to PC

try ifconfig first and take the interface name:enp2s0 (can change with PCs)

In [ ]:
sudo ip addr add 192.168.1.1/24 dev enp2s0

Define:

192.168.1.2 to UR5

Callibration the real robot, run:

In [ ]:
roslaunch ur_calibration calibration_correction.launch \
  robot_ip:=<robot_ip> target_filename:="${HOME}/my_robot_calibration.yaml"

If you calibrated your robot before, pass that calibration to the launch file:

In [ ]:
roslaunch ur_robot_driver ur5_e_bringup.launch robot_ip:=192.168.1.2 \
  kinematics_config:="${HOME}/my_robot_calibration.yaml"

You are ready to test de FK and IK

You need to install the pkg: rqt_joint_trajectory_controller

In [ ]:
sudo apt-get install ros-melodic-rqt-joint-trajectory-controller

Test the FK

In [ ]:
rosrun rqt_joint_trajectory_controller rqt_joint_trajectory_controller

Tho use moveIt!

https://github.com/ros-industrial/universal_robot/tree/melodic-devel

First launch the driver:


In [ ]:
roslaunch ur_robot_driver ur5_e_bringup.launch robot_ip:=192.168.1.2 \
  kinematics_config:="${HOME}/my_robot_calibration.yaml"

you have first to modify the file: https://github.com/ros-industrial/universal_robot/blob/kinetic-devel/ur5_e_moveit_config/config/controllers.yaml

The action_ns has to be changed to: scaled_pos_joint_traj_controller/follow_joint_trajectory

In [ ]:
roslaunch ur5_e_moveit_config ur5_e_moveit_planning_execution.launch

In [ ]:
roslaunch ur5_e_moveit_config moveit_rviz.launch config:=true

